In [18]:
import matplotlib as plt
import pandas as pd
from math import sqrt


In [30]:
era5_climate = pd.read_csv(r"era5_wefesiteanalyst_finca_turrialba.csv")
date_range = pd.date_range(start='2022-05-01 00:00:00', freq='H', periods=len(era5_climate))

# Set the date range as the index of the DataFrame
era5_climate.set_index(date_range, inplace=True)
# Remove all columns between column index 1 to 4 (4 is time)
era5_climate.drop(era5_climate.iloc[:, 0:4], inplace=True, axis=1)
era5_climate.head()

,ghi,t_air,e,tp,windspeed
2022-05-01 00:00:00,5.799167,16.698944,-0.004829,2.405666,0.154470
2022-05-01 01:00:00,0.000000,16.506805,-0.000298,2.969206,0.505367
2022-05-01 02:00:00,0.000000,16.099213,0.005770,2.260137,0.765288
2022-05-01 03:00:00,0.000000,16.630585,0.005076,1.799162,1.165331
2022-05-01 04:00:00,0.000000,16.444916,0.000417,1.140648,1.027639


In [33]:
# load local weather data file
local_weather = pd.read_csv(r"local.csv")
local_weather.head()


# Create a date range with the desired starting date and frequency
date_range = pd.date_range(start='2022-05-29 15:30:00', freq='30min', periods=len(local_weather))

# Set the date range as the index of the DataFrame
local_weather.set_index(date_range, inplace=True)
 
    

# create new dataframe only consisting out of data for required parameters


columns = ['t_air_is', 'windspeed_is', 'tp_is','ghi_is', 'e_is']
# check whether local measurement was ETO, from era5 we have retrieved evapo total, eto is also available though
values = local_weather[columns].values
sp = pd.DataFrame(values, columns=columns, index = local_weather.index)

sp.head()
print(sp.dtypes)

# convert the datatype of a column to numeric
sp = sp.apply(pd.to_numeric, errors = 'coerce')

# check the datatypes again
print(sp.dtypes)

# Group by hour
# grouped = selected_parameters.groupby(pd.Grouper(freq='H'))

# Calculate mean of each group
# local_weather_H = grouped.mean()

# Resample the 30-minute timeseries data to hourly data
local_weather_H = sp.resample('H').mean(numeric_only=True)
# convert rain  unit (sum value (required after calculating mean) and convert inch to mm)
local_weather_H['tp_is'] *= 2*25.4
# convert ET unit (sum value and convert inch to mm)
local_weather_H['e_is'] *= 2*25.4
# convert wind speed unit (mph to m/s)
local_weather_H['windspeed_is']*=0.44704 
# convert temperature unit (Fahrenheit to Celsius)
local_weather_H['t_air_is']=(local_weather_H['t_air_is']-32)/1.8
local_weather_H.head()


t_air_is        object
windspeed_is    object
tp_is           object
ghi_is          object
e_is            object
dtype: object
t_air_is        float64
windspeed_is    float64
tp_is           float64
ghi_is          float64
e_is            float64
dtype: object


,t_air_is,windspeed_is,tp_is,ghi_is,e_is
2022-05-29 15:00:00,20.000000,0.89408,1.016,13.0,0.0
2022-05-29 16:00:00,19.444444,2.90576,25.654,13.0,0.0
2022-05-29 17:00:00,19.444444,1.11760,12.954,8.5,0.0
2022-05-29 18:00:00,19.166667,0.67056,4.826,0.0,0.0
2022-05-29 19:00:00,18.888889,1.34112,2.540,0.0,0.0


In [ ]:
# cutting local and remote data to same time range
# remote data


In [35]:
print(pd.concat(['era5_climate'], ['local_weather_H'], axis=1))

C:\Users\Julian\AppData\Local\Temp\ipykernel_2716\1112579989.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  print(pd.concat(['era5_climate'], ['local_weather_H'], axis=1))


TypeError: concat() got multiple values for argument 'axis'

In [ ]:
# plot global horizontal irradiance
era5_wefe.loc[:, ['ghi']].plot(title='Irradiance')
plt.xlabel('Time')
plt.ylabel('Irradiance in $W/m^2$')

In [ ]:
# plot temperature
era5_wefe.loc[:, ['t_air']].plot(title='Temperature')
plt.xlabel('Time')
plt.ylabel('Air Temperature in $°C$')